In [1]:
import numpy as np 
import pandas as pd 
import json 
from geopy.geocoders import Nominatim 
import requests 



from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

import csv
import re
import urllib
from datetime import datetime
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
import matplotlib.colors as colors

print("Libs imported")

Libs imported


Reading postal code from Wikipedia

In [2]:
url = 'https://en.wikipedia.org/wiki/B_postcode_area' 
r = requests.get(url)
NIT = BeautifulSoup(r.content,"lxml")

table=NIT.find('table', {'class', 'wikitable sortable'})

rows = []
for row in table.find_all('tr'):
     rows.append([val.text for val in row.find_all('td')])



        
df = pd.DataFrame(rows)
df.columns = ["Town", "Coverage", "Area"]
df = df.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)

dataSet=df.drop(df.index[0])

postalRows=[]
for row in table.find_all('tr'):
     postalRows.append([val.text for val in row.find_all('th')])


df1 = pd.DataFrame(postalRows)
df1.columns = ["Postal", "None", "None","None"]

df1 = df1.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
dataSetPost=df1.drop(df.index[0])


cityData = dataSet
cityData["Postal"] = dataSetPost["Postal"]


cityData.head()



,Town,Coverage,Area,Postal
1,BIRMINGHAM,"Birmingham City Centre, Broad Street (east)",Birmingham,B1
2,BIRMINGHAM,"Birmingham City Centre, New Street",Birmingham,B2
3,BIRMINGHAM,"Birmingham City Centre, Newhall Street",Birmingham,B3
4,BIRMINGHAM,"Birmingham City Centre, Corporation Street (no...",Birmingham,B4
5,BIRMINGHAM,"Digbeth, Highgate, Lee Bank",Birmingham,B5


Getting property data

In [3]:
url = 'https://propertydata.co.uk/cities/Birmingham' 
r = requests.get(url)
NIT = BeautifulSoup(r.content,"lxml")

table=NIT.find('table')

rows = []
for row in table.find_all('tr'):
     rows.append([val.text for val in row.find_all('td')])

dfprop = pd.DataFrame(rows)
dfprop.columns = ["Postal", "AvgYield", "AvgPrice","PerSqftPrice","FiveYearChange","Links"]

dfprop = dfprop.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
dataSetPrice=dfprop.drop(dfprop.index[0])
dataSetPrice.head()

,Postal,AvgYield,AvgPrice,PerSqftPrice,FiveYearChange,Links
1,B1,4.8%,"£218,216",£317,+19%,Explore data
2,B3,4.3%,"£239,971",£286,+23%,Explore data
3,B5,5.2%,"£211,124",£258,+19%,Explore data
4,B9,5.3%,"£139,358",£134,+23%,Explore data
5,B11,4.0%,"£171,299",£143,+22%,Explore data


Cleaning property data

In [4]:
dataSetPrice.columns

Index(['Postal', 'AvgYield', 'AvgPrice', 'PerSqftPrice', 'FiveYearChange',
       'Links'],
      dtype='object')

In [5]:
dataSetPrice.drop(columns=['AvgYield', 'PerSqftPrice', 'FiveYearChange','Links'],inplace=True)
dataSetPrice['AvgPrice'] = dataSetPrice['AvgPrice'].str.replace("£","")
dataSetPrice['AvgPrice'] = dataSetPrice['AvgPrice'].str.replace(',', '')
dataSetPrice['AvgPrice'] = pd.to_numeric(dataSetPrice['AvgPrice'])



Merge average price and city data

In [6]:
#dataSetPrice
#cityData.head()



cityData['Postal'] = cityData['Postal'].str.replace(' ','')
dataSetPrice['Postal'] = dataSetPrice['Postal'].str.replace(' ','')

data=pd.merge(cityData,dataSetPrice,left_on="Postal",right_on="Postal",how="inner")

data.head()

,Town,Coverage,Area,Postal,AvgPrice
0,BIRMINGHAM,"Birmingham City Centre, Broad Street (east)",Birmingham,B1,218216
1,BIRMINGHAM,"Birmingham City Centre, Newhall Street",Birmingham,B3,239971
2,BIRMINGHAM,"Digbeth, Highgate, Lee Bank",Birmingham,B5,211124
3,BIRMINGHAM,"Bordesley Green, Bordesley",Birmingham,B9,139358
4,BIRMINGHAM,"Sparkhill, Sparkbrook, Tyseley",Birmingham,B11,171299


In [7]:
geocsv = pd.read_csv("http://burans.co/apps/geo.csv") 

data["lat"]=geocsv["Lat"]
data["lng"]=geocsv["Lng"]

data.head()

,Town,Coverage,Area,Postal,AvgPrice,lat,lng
0,BIRMINGHAM,"Birmingham City Centre, Broad Street (east)",Birmingham,B1,218216,52.478955,-1.905994
1,BIRMINGHAM,"Birmingham City Centre, Newhall Street",Birmingham,B3,239971,52.480859,-1.900922
2,BIRMINGHAM,"Digbeth, Highgate, Lee Bank",Birmingham,B5,211124,52.886993,-2.886859
3,BIRMINGHAM,"Bordesley Green, Bordesley",Birmingham,B9,139358,52.479293,-1.838810
4,BIRMINGHAM,"Sparkhill, Sparkbrook, Tyseley",Birmingham,B11,171299,52.463967,-1.875828


In [9]:
!conda install -c conda-forge folium=0.5.0 --yes

import folium


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [10]:
map_bm = folium.Map(location=[52.4862, -1.8904], zoom_start=10)

for lat, lng, borough, neighborhood in zip(data['lat'], data['lng'], data['Coverage'], data['Area']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bm)  
    


map_bm

In [15]:
CLIENT_ID = 'ASMR4VFW5S1PCAQMNNRX2E4SS3BA1AMRSS5AXFYIHBKTH3XU' 
CLIENT_SECRET = 'CKDXBOUW2YKEFPTIQGQHCAYD5Q0VJVOZEZQBMKQYB1C1KQSU' 
VERSION = '20191005'

In [16]:
def getBuiltUrl(neigh_lat,neigh_long,radius=1400):
    # type your answer here
    LIMIT=100
    #radius=1000
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_long, 
    radius, 
    LIMIT)
    return url

Forsquare API

In [18]:
Town, Coverage, Area, Postal, AvgPrice, lat, lng = data.iloc[0]
print('Latitude and longitude values of {} are {}, {}.'.format(Coverage, lat, lng))
results = requests.get(getBuiltUrl(lat,lng)).json()
results

Latitude and longitude values of Birmingham City Centre, Broad Street (east)  are 52.4789552, -1.9059943.


{'meta': {'code': 200, 'requestId': '5d991b93f9dbde00382868f0'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Birmingham',
  'headerFullLocation': 'Birmingham',
  'headerLocationGranularity': 'city',
  'totalResults': 234,
  'suggestedBounds': {'ne': {'lat': 52.49155521260001,
    'lng': -1.8853450439850952},
   'sw': {'lat': 52.46635518739999, 'lng': -1.9266435560149047}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b05882df964a520cbb622e3',
       'name': 'Birmingham Museum & Art Gallery',
       'location': {'address': 'Chamberlain Sq',
        'lat': 52.48038458300301,
        'lng': -1.9039049525886564,
        'labeledLatLngs': [{'label': 'display',
          'lat': 5

In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Forsquare API data

In [29]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Birmingham Museum & Art Gallery,History Museum,52.480385,-1.903905
1,Birmingham Town Hall,Concert Hall,52.479509,-1.903560
2,Hyatt Regency Birmingham,Hotel,52.478096,-1.909324
3,Adams,Restaurant,52.479797,-1.901482
4,Symphony Hall,Concert Hall,52.478874,-1.910229


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


100 venues were returned by Foursquare.


Find nearby values for all locations 

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        LIMIT=100
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
venues = getNearbyVenues(names=data['Coverage'], 
                                   latitudes=data['lat'], 
                                   longitudes=data['lng'], 
                                   radius=500)

Birmingham City Centre, Broad Street (east) 
Birmingham City Centre, Newhall Street 
Digbeth, Highgate, Lee Bank 
Bordesley Green, Bordesley 
Sparkhill, Sparkbrook, Tyseley 
Balsall Heath, Sparkbrook, Highgate 
Moseley, Billesley 
Kings Heath, Yardley Wood, Druids Heath, Highter's Heath, Warstock 
Edgbaston, 
Ladywood 
Harborne, Edgbaston 
Winson Green, Hockley 
Lozells, Newtown, Birchfield 
Handsworth Wood, Handsworth, Birchfield 
Erdington, Stockland Green, Short Heath, Perry Common 
Erdington, Wylde Green, Tyburn 
Yardley 
Sheldon, Yardley 
Acocks Green 
Hall Green 
Selly Oak, Bournbrook, Selly Park, Weoley Castle, California (part of) 
Bournville, Cotteridge, Stirchley 
Northfield, Longbridge, West Heath 
Woodgate, Bartley Green, Quinton, California (part of) 
Kitts Green, Stechford 
Castle Bromwich, Smith's Wood, Bromford, Hodge Hill 
Chelmsley Wood, Marston Green,  Kingshurst,  Fordbridge 
Perry Barr, Great Barr, Hamstead 
Perry Barr, Kingstanding, Great Barr 
Rednal, New Frankle

In [33]:
print(venues.shape)
venues.head()

(1108, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Birmingham City Centre, Broad Street (east)",52.478955,-1.905994,Birmingham Town Hall,52.479509,-1.903560,Concert Hall
1,"Birmingham City Centre, Broad Street (east)",52.478955,-1.905994,Birmingham Museum & Art Gallery,52.480385,-1.903905,History Museum
2,"Birmingham City Centre, Broad Street (east)",52.478955,-1.905994,Hyatt Regency Birmingham,52.478096,-1.909324,Hotel
3,"Birmingham City Centre, Broad Street (east)",52.478955,-1.905994,Symphony Hall,52.478874,-1.910229,Concert Hall
4,"Birmingham City Centre, Broad Street (east)",52.478955,-1.905994,Post Office Vaults,52.479233,-1.901960,Pub


Count neigbourhood 

In [35]:
venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acocks Green,10,10,10,10,10,10
"Balsall Heath, Sparkbrook, Highgate",15,15,15,15,15,15
"Birmingham City Centre, Broad Street (east)",75,75,75,75,75,75
"Birmingham City Centre, Newhall Street",79,79,79,79,79,79
"Boldmere, New Oscott, Wylde Green",13,13,13,13,13,13
"Bordesley Green, Bordesley",4,4,4,4,4,4
"Bournville, Cotteridge, Stirchley",12,12,12,12,12,12
Bromsgrove (east),79,79,79,79,79,79
"Castle Bromwich, Smith's Wood, Bromford, Hodge Hill",81,81,81,81,81,81


In [36]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))


There are 131 uniques categories.


Analyzing Neighborhood Location (Each)


In [39]:
bm_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

bm_onehot['Neighborhood'] = venues['Neighborhood'] 

fixed_columns = [bm_onehot.columns[-1]] + list(bm_onehot.columns[:-1])
bm_onehot = bm_onehot[fixed_columns]

bm_onehot.head()

,Neighborhood,Aquarium,Art Gallery,Asian Restaurant,Auto Garage,Bakery,Bar,Beer Bar,Betting Shop,Bistro,...,Toy / Game Store,Track Stadium,Trail,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Yoga Studio
0,"Birmingham City Centre, Broad Street (east)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Birmingham City Centre, Broad Street (east)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Birmingham City Centre, Broad Street (east)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Birmingham City Centre, Broad Street (east)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Birmingham City Centre, Broad Street (east)",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
bm_onehot.shape


(1108, 132)

Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [42]:
bm_grouped = bm_onehot.groupby('Neighborhood').mean().reset_index()
bm_grouped

,Neighborhood,Aquarium,Art Gallery,Asian Restaurant,Auto Garage,Bakery,Bar,Beer Bar,Betting Shop,Bistro,...,Toy / Game Store,Track Stadium,Trail,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Yoga Studio
0,Acocks Green,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.100000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.100000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000
1,"Balsall Heath, Sparkbrook, Highgate",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.066667,0.000000,0.000000,0.0,0.000000,0.000000
2,"Birmingham City Centre, Broad Street (east)",0.000000,0.026667,0.000000,0.0,0.013333,0.080000,0.013333,0.000000,0.026667,...,0.000000,0.0,0.013333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,"Birmingham City Centre, Newhall Street",0.000000,0.012658,0.012658,0.0,0.012658,0.050633,0.012658,0.000000,0.037975,...,0.012658,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.012658,0.012658
4,"Boldmere, New Oscott, Wylde Green",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,"Bordesley Green, Bordesley",0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,"Bournville, Cotteridge, Stirchley",0.000000,0.000000,0.000000,0.0,0.083333,0.083333,0.000000,0.000000,0.000000,...,0.000000,0.0,0.083333,0.083333,0.000000,0.000000,0.000000,0.0,0.083333,0.000000
7,Bromsgrove (east),0.000000,0.000000,0.012658,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.012658,0.0,0.000000,0.000000
8,"Castle Bromwich, Smith's Wood, Bromford, Hodge...",0.000000,0.012346,0.000000,0.0,0.012346,0.037037,0.000000,0.000000,0.037037,...,0.012346,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.012346,0.012346
9,"Chelmsley Wood, Marston Green, Kingshurst, F...",0.000000,0.000000,0.000000,0.0,0.000000,0.200000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [43]:
a=bm_grouped
a.head()

,Neighborhood,Aquarium,Art Gallery,Asian Restaurant,Auto Garage,Bakery,Bar,Beer Bar,Betting Shop,Bistro,...,Toy / Game Store,Track Stadium,Trail,Train Station,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Shop,Yoga Studio
0,Acocks Green,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.100000,0.0,0.000000,...,0.000000,0.0,0.000000,0.1,0.000000,0.0,0.0,0.1,0.000000,0.000000
1,"Balsall Heath, Sparkbrook, Highgate",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.066667,0.0,0.0,0.0,0.000000,0.000000
2,"Birmingham City Centre, Broad Street (east)",0.0,0.026667,0.000000,0.0,0.013333,0.080000,0.013333,0.0,0.026667,...,0.000000,0.0,0.013333,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
3,"Birmingham City Centre, Newhall Street",0.0,0.012658,0.012658,0.0,0.012658,0.050633,0.012658,0.0,0.037975,...,0.012658,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.012658,0.012658
4,"Boldmere, New Oscott, Wylde Green",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000


Top 5 common venues in Neighbourhoods

In [44]:
num_top_venues = 5

for hood in a['Neighborhood']:
    print("----"+hood+"----")
    temp = a[a['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Acocks Green ----
             venue  freq
0            Hotel   0.1
1              Pub   0.1
2  Warehouse Store   0.1
3      Supermarket   0.1
4         Beer Bar   0.1


----Balsall Heath, Sparkbrook, Highgate ----
                  venue  freq
0     Indian Restaurant  0.47
1              Tea Room  0.07
2   Moroccan Restaurant  0.07
3  Pakistani Restaurant  0.07
4  Fast Food Restaurant  0.07


----Birmingham City Centre, Broad Street (east) ----
                venue  freq
0                 Pub  0.11
1         Coffee Shop  0.09
2                 Bar  0.08
3        Cocktail Bar  0.07
4  Italian Restaurant  0.07


----Birmingham City Centre, Newhall Street ----
                venue  freq
0         Coffee Shop  0.11
1                 Pub  0.10
2  Italian Restaurant  0.05
3                 Bar  0.05
4   Indian Restaurant  0.04


----Boldmere, New Oscott, Wylde Green ----
                venue  freq
0   Indian Restaurant  0.38
1                 Pub  0.15
2          Steakhouse  0.08
3  

           venue  freq
0            Pub  0.16
1    Supermarket  0.11
2  Grocery Store  0.11
3    Coffee Shop  0.11
4    Pizza Place  0.05


----Tamworth, Wilnecote, Amington ----
            venue  freq
0     Coffee Shop  0.11
1             Pub  0.09
2            Café  0.05
3   Shopping Mall  0.05
4  Clothing Store  0.05


----Walmley, Minworth ----
                        venue  freq
0                 Sports Club  0.25
1                 Rugby Pitch  0.25
2     Health & Beauty Service  0.25
3  Construction & Landscaping  0.25
4                    Aquarium  0.00


----Water Orton, Coleshill, Nether Whitacre ----
               venue  freq
0      Grocery Store  0.25
1                Pub  0.25
2  Fish & Chips Shop  0.25
3      Train Station  0.25
4           Aquarium  0.00


----West Bromwich (south, west and town centre) ----
            venue  freq
0     Coffee Shop  0.11
1             Pub  0.09
2            Café  0.05
3   Shopping Mall  0.05
4  Clothing Store  0.05


----Winson Green, 